## пороговая подпись на изогениях
scheme_setup() - генерация параметров схемы
        
key_generation(k, EK, PM, PS, QS) - генерация ключей и их распределение
            
choose_signers(s) - подписывающие
sign_by_group(mes, sk, gpk, e, f, p, PM, PS, QS, EK, s, signers, k, table) - групповая подпись сообщения
            
verify_sign(mes,sign,gpk[k+1],e,p,PM,deg) - проверка сгенерированной подписи для нашего сообщения

In [1]:
### import shutil
import numpy as np
import scipy.special
import random
from sys import exit
from itertools import product


class TSS:
    def __init__(self, n=10, t=5):
        self.lam = 4
        self.l = [2, 3, 5]
        self.n = n
        self.t = t
    
#     генерация характеристики поля
    @staticmethod
    def __gen_p(l, lam):
        e = [0, 0, 0]
        ll = 1
        for i in range(len(l)):
            e[i] = lam / 2
            while True:
                if l[i] ^ e[i] >= 2 ^ (2 * lam):
                    ll = ll * l[i] ^ e[i]
                    break
                else:
                    e[i] += 1
        f = l[2] + 1
        while True:
            p1 = ll * f + 1
            p2 = ll * f - 1
            if ((p1 in Primes())==True ) and ((f in Primes())==True ):
                p = p1
                break
            if ((p2 in Primes())==True ) and ((f in Primes())==True ):
                p = p2
                break
            f = next_prime(f)
        return e, f, p
    
#     генерация кривой
    @staticmethod
    def __gen_supersingularEC(p):
        if mod(p,4)==3:
            print("mod(p,4) == 3")
            E = EllipticCurve(GF(p),[0,0,0,-1,0])
        else:
            D = gen_D(p)
            #print "D = ",D
            H = hilbert_class_polynomial(-D)
            #print "H: ",H
            js = H.roots(GF(p**2),multiplicities=False)
            #print "js = ",js
            F = GF(p**2)
            j = js[0]
            tmp = (4 * (1728-j) )
            A = F((27*j) * pow(tmp,-1,p**2) )
            E = EllipticCurve(GF(p**2),[A,-A])
        return E
    
    @staticmethod
    def __gen_base_point(E, l, e):
        N = E.cardinality()
        F = E.base_field()
        PP = E.random_point()        
        c = ZZ(N / (l^e)^2)
        P = c * PP
        checkP = ZZ(l^e)*P
        if checkP[0] == 0 and checkP[2] == 0:
            PA = P
            return PA
        return None
        
#     генерация точки Ps, Qs
    def __gen_bases(self, E, l, e):
        while True:
            P = self.__gen_base_point(E,l,e)
            Q = self.__gen_base_point(E,l,e)
            ee = P.weil_pairing(Q, ZZ(l**e))
            if ee^(l^e) == 1:
                return P,Q
    
#     генерация параметров схемы
    def scheme_setup(self):
        print("[*] Генерация параметров схемы ...")
        e, f, p = self.__gen_p(self.l, self.lam)
        print("p = ", p)
        E = self.__gen_supersingularEC(p)
        print("E = ", E)
        P = E.random_point()
        F.<a> = GF(p^2)
        ff = a.minpoly('x')
        K.<t> = GF(p^2,modulus=ff)
        EK = E.base_extend(K)
        N = EK.cardinality()
        print("N = ",factor(N))
        PS,QS = self.__gen_bases(EK, self.l[2], e[2])
        print("\nPs = {}\nQs = {}\n".format(PS, QS))
        PP = self.__gen_base_point(EK, self.l[2], e[2])
        if PP[0] != PS[0] and PP[0] != QS[0]:
            PM=PP
        else:
            raise ValueError("Ошибка генерации точки PM")
        print("\nPS = ",PS)
        print("QS = ",QS)
        print("PM = ",PM)
        print("====================================================\n")
        return e,f,p,EK,PM,PS,QS

######## генерация ключей ######### 
    @staticmethod
    def gen_base_point(E,l,e):
        N=E.cardinality()
        F=E.base_field()
        PP=E.random_point()
        c=ZZ(N/(l^e)^2)
        P=c*PP
        checkP=ZZ(l^e)*P
        if checkP[0]==0 and checkP[2]==0:
            PA=P
            return PA
        return None

    
    def gen_bases(self, E,l,e):
        while True:
            P=self.gen_base_point(E,l,e)
            Q=self.gen_base_point(E,l,e)
            ee=P.weil_pairing(Q, ZZ(l**e))
            if ee^(l^e)==1:
                return P,Q

    @staticmethod
    def gen_kernel_coefficients(l,e):
        while True:
            m = randint(1,l^e-1)
            n = randint(1,l^e-1)
            if mod(m,l^e) != 0 and mod(n,l^e) != 0:
                return m,n
    
    @staticmethod
    def comb(k, n):
        """Генерация сочетаний из `n` по `k` без повторений."""
        d = list(range(0, k))
        yield d
        while True:
            i = k - 1
            while i >= 0 and d[i] + k - i + 1 > n:
                i -= 1
            if i < 0:
                return
            d[i] += 1
            for j in range(i + 1, k):
                d[j] = d[j - 1] + 1
            yield d

    
    def key_distribution(self, k,t,n):
        table = [2**n-1] * k
        i = 0
        for c in self.comb(t-1,n):
            for j in c:
                table[i] = table[i]^^(1<<j)
            i+=1
        return table 

    
    def gen_users_data(self, EK,PM,k,num,ll,ee,ms,ns,table):
        gpk = list()
        sk = list()
        phi = list()
        deg = 1
        j = randint(1,10^10)
        if mod(j,2)==0:
            P,Q = self.gen_bases(EK,ll[0],ee[0])
            m,n = self.gen_kernel_coefficients(ll[0],ee[0])
        else:
            P,Q = self.gen_bases(EK,ll[1],ee[1])
            m,n = self.gen_kernel_coefficients(ll[1],ee[1])
        for i in range(num):
            sk.append(dict())
            sk[i]['s'] = [ms,ns]
            sk[i]['ver'] = [m,n]
        phi.append(EllipticCurveIsogeny(EK, m*P+n*Q))
        print(" isogeny phi-ver: ",phi[0])
        E = phi[0].codomain()
        deg = deg*phi[0].degree()
        print("deg phi-ver = ",factor(phi[0].degree()))
        gpk.append([E,P,Q])
        imagePM = phi[0](PM)
        PM = imagePM
        EK = E
        for i in range(0,k):
            print("____________________")
            print()
            print("phi ",i)
            j = randint(1, 10^10)
            if mod(j,2) == 0:
                P,Q = self.gen_bases(EK,ll[0],ee[0])
                m,n = self.gen_kernel_coefficients(ll[0],ee[0])
            else:
                P,Q = self.gen_bases(EK,ll[1],ee[1])
                m,n = self.gen_kernel_coefficients(ll[1],ee[1])
            for j in range(num):
                if table[i]&(1<<j):
                    sk[j][i] = [m,n]
            print("m = ",m,", n = ",n)
            phi.append(EllipticCurveIsogeny(EK, m*P+n*Q))
            print(i," isogeny phi: ",phi[i+1])
            E = phi[i+1].codomain()
            deg = deg*phi[i+1].degree()
            print("deg phi = ",factor(phi[i+1].degree()))
            gpk.append([E,P,Q])
            imagePM = phi[i+1](PM)
            PM = imagePM
            EK = E
        return gpk,sk,PM,deg

    def key_generation(self, k, EK, PM, PS, QS):
        print("[*] Генерация и распределение ключей ...")
        mS,nS = self.gen_kernel_coefficients(self.l[2],e[2])
        print("ms = ",mS,", ns = ",nS)
        S = mS*PS+nS*QS
        print("S = ",S)
        phiS = EllipticCurveIsogeny(EK,S)
        print("phiS = ",phiS)
        ES = phiS.codomain()
        table = self.key_distribution(k, self.t, self.n)
        #print table
        gpk,sk,imagePM,deg = self.gen_users_data(EK,PM,k,self.n,self.l,e,mS,nS,table)
        print("_____________")
        gpk.append([ES,imagePM])
        print("deg: ",deg)
        print("\n",90 * "=","\n----- Открытые ключи -----")
        for i in gpk:
            print(i)
        j = 0
        print("\n",90 * "=","\n----- Секретные ключи -----")
        for i in sk:
            print("User",j,". ",i)
            j+=1
            print(90 * "-")
        return gpk, sk, deg, table
    

######## генерация подписи 
# выбор подписывающих
    def choose_signers(self, s):
        count = random.sample(range(self.n), self.n)
        count.remove(s)
        values = list()
        values.append(s)
        for i in range (self.t-1):
            val = count.pop()
            values.append(val)
        return values
    
    def __sign_by_user(self, e, f, p, QQ, QMI, PS, QS, sk, sk_pr, pk, E_prev):
        m,n = sk[0],sk[1]
        mS,nS = sk_pr[0],sk_pr[1]
        P,Q = pk[1],pk[2]
        phi = E_prev.isogeny(m*P+n*Q)
        #E_user = phi.codomain()
        ksi = E_prev.isogeny(mS*PS+nS*QS)
        imageP = ksi(P)
        imageQ = ksi(Q)
        E_s = ksi.codomain()
        imageQM = phi(QMI)
        imagePS = phi(PS)
        imageQS = phi(QS)
        ksi_is = E_s.isogeny(m*imageP+n*imageQ)
        E_is = ksi_is.codomain()
        sigma = [E_is,QQ,imageQM,imagePS,imageQS]
        return sigma
    
    def __choose_signer(self, n_isog, signers, table):
        i = 0
        while 1:
            if table[n_isog] & (1<<signers[i]):
                break
            i+=1
        return signers[i]
    
    def __sign_by_himself(self, e, f, p, QQ, QMI, PS, QS, sk, sk_pr, pk, E_prev):
        m,n = sk[0],sk[1]
        #mS,nS = sk_pr[0],sk_pr[1]
        P,Q = pk[1],pk[2]
        phi = E_prev.isogeny(m*P+n*Q)
        #E_user = phi.codomain()
        imageQM = phi(QMI)
        imagePS = phi(PS)
        imageQS = phi(QS)
        sigma = [QQ,imageQM,imagePS,imageQS]
        return sigma  
    
    def sign_by_group(self, mes, sk, pk, e, f, p, PM, PS, QS, EK, s, signers, k, table):
        print("[*] Начало подписывания сообщения " + str(mes) + " пользователя " + str(s) + " ...")
        print("signers: ", signers)
        h = hash(mes)
        print("hash ",h)
        PP = self.__gen_base_point(EK, self.l[2], e[2])
        if PP[0] != PS[0] and PP[0] != QS[0] :
            QQ = PP
            QM = ZZ(h)*QQ
            print("QM = {}\nПорядок QM = {}\n\n".format(QM, QM.order()))
        else:
            raise ValueError("Ошибка генерации точки QM")
    
        E_prev = EK
        imagePS = PS
        imageQS = QS
        imageQM = QM
        
        sigma = self.__sign_by_himself(e,f,p,QQ,imageQM,imagePS,imageQS,sk[s]['ver'],sk[s]['s'],gpk[0],E_prev)
        QQ, imageQM, imagePS, imageQS = sigma[0],sigma[1], sigma[2],sigma[3]
        print("Signer verification")
        
        for i in range(k):
            sig = self.__choose_signer(i, signers, table)
            if sig != s:
                print("___________________")
                print("Sending to user ",sig)
                sigma = self.__sign_by_user(e,f,p,QQ,imageQM,imagePS,imageQS,sk[sig][i],sk[sig]['s'],gpk[i+1],gpk[i][0])
                E_is,QQ,imageQM,imagePS,imageQS = sigma[0],sigma[1], sigma[2],sigma[3],sigma[4]
                #E_prev = gpk[i+1][0]
                #proof
                phi_proof = gpk[i+1][0].isogeny(sk[s]['s'][0]*imagePS+sk[s]['s'][1]*imageQS)
                E_tmp = phi_proof.codomain()
                print("Check knowledge of isogeny phiS:")
                if E_tmp != E_is:
                    print("!ALARM! User ",sig," doesn't know secret isogeny!!!")
                else:
                    print("E_tmp = E_is")
            else:
                print("___________________")
                print("Sign by ",s)
                sigma = self.__sign_by_himself(e,f,p,QQ,imageQM,imagePS,imageQS,sk[s][i],sk[s]['s'],gpk[i+1],gpk[i][0])
                QQ,imageQM,imagePS,imageQS = sigma[0],sigma[1], sigma[2],sigma[3]
            print("Sigma[",i,"] = ",imageQM,imagePS,imageQS)
        
        threshold_sigma = []
        threshold_sigma.append(QQ)
        threshold_sigma.append(imageQM)
        print("Пороговая подпись: ", threshold_sigma)
#         print("*********** time: **********")
#         print(t_new-t_old)
#         print("****************************")
        return threshold_sigma
        
######## проверка подписи #######     
    def verify_sign(self, mes,sign,gpk,e,p,PM,deg):
        print("[*] Проверка подписи сообщения ...")
        h=hash(mes)
        print("message hash: ",h)
        QQ, imageQM = sign[0], sign[1]
        QM = ZZ(h)*QQ
        ES, imagePM = gpk[0], gpk[1]
        e1 = imagePM.weil_pairing(imageQM, ZZ(self.l[2]^e[2]))
        print("pairing e1 = ",e1)
        e2 = (PM).weil_pairing(QM,ZZ(self.l[2]^e[2]))
        print("pairing e2 = ",e2^deg)
        return e1 == e2^deg
        

    
    
def start():
    print("====================================================")
    print("Выберите действие: ")
    print("1. Сгенерировать открытые параметры")
    print("2. Сгенерировать ключи")
    print("3. Сгенерировать подпись")
    print("4. Проверить подпись")
    print("5. Печать меню")
    print("Выход (любое другое число)")
    print("====================================================")
    
    
    
if __name__ == "__main__":
    tss = None
    EK = PM = PS = QS = None
    print("1. Введите n и t \n\
2. Если хотите использовать значения по умолчанию n=10 t=5")
    choose = int(input())
    if choose == 1:
        n = int(input("введите n "))
        t = int(input("введите t "))
        tss = TSS(n, t)
        print(tss.n, tss.t)
    elif choose == 2:
        tss = TSS()
        print(tss.n, tss.t)
        n, t = 10, 5
    else:
        print("Ошибка!")
        exit()
    print("Параметр безопасности lambda = ", tss.lam)
    start()
    while 1:
        choose = int(input("\nвыберите действие (меню - 5)"))
        if choose == 1:
            e, f, p, EK, PM, PS, QS = tss.scheme_setup()
#             print("\n\n{}\n\n".format(e))
        elif choose == 2:
            k = scipy.special.comb(n, t - 1, exact=True)
            gpk, sk, deg, table = tss.key_generation(k, EK, PM, PS, QS)
        elif choose == 3:
            s = int(input("Введите ваше число (инициатора подписи): "))
            signers = tss.choose_signers(s) # подписывающие 
            print(signers)
            
            print("Введите сообщение для подписи: ")
            mes = input()
            sign = tss.sign_by_group(mes, sk, gpk, e, f, p, PM, PS, QS, EK, s, signers, k, table)
            
        elif choose == 4:
            print("Введите сообщение: ")
            mes = input()
            result_verify = tss.verify_sign(mes,sign,gpk[k+1],e,p,PM,deg)
            print ("Подпись для сообщения ",mes," ", result_verify)
        elif choose == 5:
            start()
        else:
            print("Выход")
            break
        

1. Введите n и t 
2. Если хотите использовать значения по умолчанию n=10 t=5
1
введите n 4
введите t 2
4 2
Параметр безопасности lambda =  4
Выберите действие: 
1. Сгенерировать открытые параметры
2. Сгенерировать ключи
3. Сгенерировать подпись
4. Проверить подпись
5. Печать меню
Выход (любое другое число)

выберите действие (меню - 5)1
[*] Генерация параметров схемы ...
p =  1516319999
mod(p,4) == 3
E =  Elliptic Curve defined by y^2 = x^3 + 1516319998*x over Finite Field of size 1516319999
N =  2^16 * 3^12 * 5^8 * 13^2

Ps = (699074679*t + 536250445 : 1313901824*t + 1372232857 : 1)
Qs = (263733089*t + 128959003 : 922650995*t + 741773153 : 1)


PS =  (699074679*t + 536250445 : 1313901824*t + 1372232857 : 1)
QS =  (263733089*t + 128959003 : 922650995*t + 741773153 : 1)
PM =  (984494066*t + 1292473236 : 1042874537*t + 1323215015 : 1)


выберите действие (меню - 5)2
[*] Генерация и распределение ключей ...
ms =  489 , ns =  477
S =  (62332477*t + 1456179747 : 691148463*t + 695112928 : 1)